In [1]:
# pip install ffmpeg

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import dash
from dash import dcc, html
import dash.exceptions
from dash.dependencies import Input, Output, State
import plotly.express as px
import geopandas as gpd
import calendar
import time
import random
from geopy.geocoders import ArcGIS
import folium
import plotly.graph_objs as go


In [3]:
file_path = 'Real_Estate_Sales_2001-2021_GL.csv'
df = pd.read_csv(file_path)
df.head(5)

/var/folders/xw/6y1gg0b108956d53nhdcltr00000gn/T/ipykernel_42685/4022399427.py:2: DtypeWarning: Columns (8,9,10,11,12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


,Serial Number,List Year,Date Recorded,Town,Address,Assessed Value,Sale Amount,Sales Ratio,Property Type,Residential Type,Non Use Code,Assessor Remarks,OPM remarks,Location
0,2020177,2020,04/14/2021,Ansonia,323 BEAVER ST,133000.0,248400.0,0.5354,Residential,Single Family,NaN,NaN,NaN,POINT (-73.06822 41.35014)
1,2020225,2020,05/26/2021,Ansonia,152 JACKSON ST,110500.0,239900.0,0.4606,Residential,Three Family,NaN,NaN,NaN,NaN
2,2020348,2020,09/13/2021,Ansonia,230 WAKELEE AVE,150500.0,325000.0,0.4630,Commercial,NaN,NaN,NaN,NaN,NaN
3,2020090,2020,12/14/2020,Ansonia,57 PLATT ST,127400.0,202500.0,0.6291,Residential,Two Family,NaN,NaN,NaN,NaN
4,200500,2020,09/07/2021,Avon,245 NEW ROAD,217640.0,400000.0,0.5441,Residential,Single Family,NaN,NaN,NaN,NaN


In [4]:
# df.isnull().sum()

In [5]:
# df['Residential Type'].unique()

In [6]:
# df['Property Type'].unique()

In [7]:
# df['OPM remarks'].unique()

In [8]:
# df.columns

In [9]:
# df.info()

In [10]:
df['Property Type'].replace({'Two Family': 'Residential', 'Single Family': 'Residential', 'Three Family': 'Residential', 'Four Family': 'Residential'}, inplace=True)

In [11]:
app = dash.Dash(__name__)


In [12]:
dropdown_columns = ['Town', 'Property Type']

# Fill only the specified dropdown columns with 'na'
df[dropdown_columns] = df[dropdown_columns].fillna('na')


# Randomly select one value from Town column
default_values = {column: random.choice(df[column].unique()) if column == 'Town' else df[column].unique().tolist() for column in dropdown_columns}



In [13]:
app.layout = html.Div([
    html.H1("Real Estate Dashboard", style={'textAlign': 'center', 'color': '#7FDBFF'}),
    # html.H2("Group Number: 6, Names: Ashmita Pal, Hrishikesh Keswani, Aakash Mehta, Nirmit Sachde", style={'textAlign': 'center', 'color': '#0074D9'}),

    html.H3("Town wise analysis", style={'textAlign': 'center', 'color': '#0074D9'}),
    html.Div([
        html.Label("Select Town:", style={'marginRight': '2em'}),
        dcc.Dropdown(
            id='town-dropdown',
            options=[{'label': town, 'value': town} for town in df['Town'].unique()],
            value='All',
            style={'width': '200px'}
        )
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginBottom': '2em'}),

    html.Div([
        html.Div([dcc.Graph(id='sales-over-years')], style={'width': '33%', 'display': 'inline-block'}),
        html.Div([dcc.Graph(id='count-of-sales')], style={'width': '33%', 'display': 'inline-block'}),
        html.Div([dcc.Graph(id='average-sale-ratio')], style={'width': '33%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

    html.H3("Year based analysis", style={'textAlign': 'center', 'color': '#0074D9'}),
    html.Div([
        html.Label("Select Year:", style={'marginRight': '2em'}),
        dcc.Dropdown(
            id='year-dropdown',
            options=[{'label': year, 'value': year} for year in df['List Year'].unique()],
            value='All',
            style={'width': '200px'}
        )
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginBottom': '2em'}),

    html.Div([
        html.Div([dcc.Graph(id='sales-over-months')], style={'width': '50%', 'display': 'inline-block'}),
        html.Div([dcc.Graph(id='count-of-sales-months')], style={'width': '50%', 'display': 'inline-block'})
    ], style={'display': 'flex', 'justifyContent': 'space-between'}),

     

    
    html.H3("Choropleth Maps", style={'textAlign': 'center', 'color': '#0074D9'}),
    html.Div([
        html.Label('Select Year:', style={'marginRight': '2em'}),
        dcc.Dropdown(
            id='year-dropdown-choropleth',
            options=[{'label': year, 'value': year} for year in df['List Year'].unique()],
            value='All',
            style={'width': '200px'}
        ),
        dcc.Graph(
            id='choropleth-map',
            config={'displayModeBar': False},
            style={'height': '50vh', 'width': '100%'}
        ),
        html.Div(id='sales-info')
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginBottom': '0.2em'}),

    html.Div([
        html.Label('Select Year:', style={'marginRight': '2em'}),
        dcc.Dropdown(
            id='year-dropdown-amount',
            options=[{'label': year, 'value': year} for year in df['List Year'].unique()],
            value='All',
            style={'width': '200px'}
        ),
        dcc.Graph(
            id='choropleth-map-amount',
            config={'displayModeBar': False},
            style={'height': '70vh', 'width': '100%'}
        ),
        html.Div(id='sales-info-amount')
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginBottom': '2em'}),

    html.H3("Property Type sales year wise", style={'textAlign': 'center', 'color': '#0074D9'}),
    html.Div([
        html.Label("Select Year:", style={'marginRight': '2em'}),
        dcc.Dropdown(
            id='year-dropdown-property',
            options=[{'label': year, 'value': year} for year in df['List Year'].unique()],
            value='All',
            style={'width': '200px'}
        )
    ], style={'display': 'flex', 'alignItems': 'center', 'justifyContent': 'center', 'marginBottom': '2em'}),

    html.Div([
        dcc.Graph(id='sales-over-property-types')
    ], style={'marginBottom': '2em'}),
    
    html.H3("Pie Chart Distribution", style={'textAlign': 'center', 'color': '#0074D9'}),
    
    html.Div([
], style={'fontFamily': 'Helvetica', 'padding': '2em'}),
    
    html.Div([
        dcc.Graph(id='property-types-distribution')
    ], style={'marginBottom': '2em'}),
    html.H3("Town sales - Property types"),
    html.Div([
        html.Label("Select Town(s):"),
        dcc.Dropdown(
            id='town-dropdown-last',
            options=[{'label': town, 'value': town} for town in df['Town'].unique()],
            multi=False,
            value=default_values['Town']
        ),
        html.Label("Select Property Type(s):"),
        dcc.Dropdown(
            id='property-type-dropdown',
            options=[{'label': prop_type, 'value': prop_type} for prop_type in df['Property Type'].unique()],
            multi=True,
            value=default_values['Property Type']
        ),
        html.Div(id='output-graph')
    ])

], style={'fontFamily': 'Helvetica', 'padding': '2em'})


In [14]:
@app.callback(
    Output('sales-over-years', 'figure'),
    [Input('town-dropdown', 'value')]
)
def update_sales_over_years(selected_town):
    if selected_town == 'All':
        data = df.groupby('List Year')['Sale Amount'].sum().reset_index()
        title = "Total Sales Over Years"
    else:
        data = df[df['Town'] == selected_town].groupby('List Year')['Sale Amount'].sum().reset_index()
        title = f"Total Sales Over Years in {selected_town}"
    
    fig = px.line(data, x='List Year', y='Sale Amount', title=title)
    return fig


In [15]:

@app.callback(
    Output('count-of-sales', 'figure'),
    [Input('town-dropdown', 'value')]
)
def update_count_of_sales(selected_town):
    if selected_town == 'All':
        data = df.groupby('List Year').size().reset_index(name='Count of Sales')
        title = "Count of Sales Over Years"
    else:
        data = df[df['Town'] == selected_town].groupby('List Year').size().reset_index(name='Count of Sales')
        title = f"Count of Sales Over Years in {selected_town}"
    
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=data['List Year'], y=data['Count of Sales'], mode='lines', name='Count of Sales'))
    
    fig.update_layout(
        title=title,
        xaxis_title='Year',
        yaxis_title='Count of Sales',
        yaxis=dict(
            tickformat=',.0f'  # Display y-axis labels in comma-separated format
        )
    )
    
    return fig


In [16]:
@app.callback(
    Output('average-sale-ratio', 'figure'),
    [Input('town-dropdown', 'value')]
)
def update_average_sale_ratio(selected_town):
    if selected_town == 'All':
        data = df.groupby('List Year')['Sales Ratio'].mean().reset_index()
        title = "Average Sale Ratio Over the Years"
    else:
        data = df[df['Town'] == selected_town].groupby('List Year')['Sales Ratio'].mean().reset_index()
        title = f"Average Sale Ratio Over the Years in {selected_town}"
    
    fig = px.line(data, x='List Year', y='Sales Ratio', title=title)
    return fig


In [17]:
df['Date Recorded'] = pd.to_datetime(df['Date Recorded'])

@app.callback(
    Output('sales-over-months', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_sales_over_months(selected_year):
    if selected_year == 'All':
        data = df.groupby(df['Date Recorded'].dt.month)['Sale Amount'].sum().reset_index(name='Sales')
        title = "Sales Over Months"
    else:
        data = df[df['List Year'] == selected_year].groupby(df['Date Recorded'].dt.month)['Sale Amount'].sum().reset_index(name='Sales')
        title = f"Sales Over Months in {selected_year}"
    
    fig = px.line(data, x='Date Recorded', y='Sales', title=title)
    fig.update_xaxes(tickvals=list(range(1, 13)), ticktext=[calendar.month_abbr[i] for i in range(1, 13)])
    return fig


In [18]:
@app.callback(
    Output('count-of-sales-months', 'figure'),
    [Input('year-dropdown', 'value')]
)
def update_count_of_sales_months(selected_year):
    if selected_year == 'All':
        data = df.groupby(df['Date Recorded'].dt.month).size().reset_index(name='Count of Sales')
        title = "Count of Sales Over Months"
    else:
        data = df[df['List Year'] == selected_year].groupby(df['Date Recorded'].dt.month).size().reset_index(name='Count of Sales')
        title = f"Count of Sales Over Months in {selected_year}"
    
    fig = px.line(data, x='Date Recorded', y='Count of Sales', title=title)
    fig.update_xaxes(tickvals=list(range(1, 13)), ticktext=[calendar.month_abbr[i] for i in range(1, 13)])
    return fig


In [19]:
@app.callback(
    Output('sales-over-property-types', 'figure'),
    [Input('year-dropdown-property', 'value')]
)
def update_sales_over_property_types(selected_year):
    if selected_year == 'All':
        data = df.groupby(['List Year', 'Property Type']).size().reset_index(name='Count of Sales')
        title = "Count of Sales Over Property Types"
    else:
        data = df[df['List Year'] == selected_year].groupby(['List Year', 'Property Type']).size().reset_index(name='Count of Sales')
        title = f"Count of Sales Over Property Types in {selected_year}"
    
    fig = px.bar(data, x='Property Type', y='Count of Sales', color='Property Type', title=title)
    return fig


In [20]:
shp = gpd.read_file("/Users/nirmitsachde/Library/CloudStorage/GoogleDrive-sachde.n@husky.neu.edu/My Drive/DAE/CVA/Project 4/tl_2019_09_cousub/tl_2019_09_cousub.shp")
# merged_data = pd.merge(shp, df, left_on='NAME', right_on='Town', how='left')

In [21]:
# merged_data


In [22]:
ct_shp = shp[shp['STATEFP'] == '09']

# Get unique years from the dataframe and sort them in ascending order
unique_years = sorted(df['List Year'].unique())

# Populate dropdown options with sorted unique years
dropdown_options = [{'label': str(year), 'value': year} for year in unique_years]
dropdown_options.append({'label': 'All years', 'value': 'All'})

@app.callback(
    [Output('choropleth-map', 'figure'),
     Output('sales-info', 'children')],
    [Input('year-dropdown-choropleth', 'value')]
)
def update_choropleth_map(selected_year):
    # Filter the dataframe based on selected year
    if selected_year == 'All':
        df_filtered = df
    else:
        df_filtered = df[df['List Year'] == selected_year]
    
    # Get total count of sales
    total_sales_count = len(df_filtered)
    
    # Group the filtered dataframe by town and count the number of sales
    sales_count = df_filtered.groupby('Town').size().reset_index(name='Sales Count')
    
    # Merge sales count with Connecticut shapefile data
    merged_data = ct_shp.merge(sales_count, how='left', left_on='NAME', right_on='Town')
    
    # Plot the choropleth map
    fig = px.choropleth(merged_data, 
                        geojson=merged_data.geometry, 
                        locations=merged_data.index, 
                        color='Sales Count',
                        hover_name='NAME',
                        title='Real Estate Sales Count by Town in Connecticut',
                        projection="mercator")
    
    # Update map layout to focus on Connecticut
    fig.update_geos(fitbounds="locations", visible=False)
    
    # Set the size of the plot
    fig.update_layout(height=600, width=800)  # Adjust the height and width as needed
    
    # Prepare sales information to display
    sales_info = f'Total Sales Count for {selected_year}: {total_sales_count}'
    
    return fig, sales_info


In [23]:
@app.callback(
    [Output('choropleth-map-amount', 'figure'),
     Output('sales-info-amount', 'children')],
    [Input('year-dropdown-amount', 'value')]
)
def update_choropleth_map_amount(selected_year):
    if selected_year == 'All':
        filtered_df = df
    else:
        filtered_df = df[df['List Year'] == selected_year]
    
    total_amount = filtered_df['Sale Amount'].sum()
    
    sales_by_town = filtered_df.groupby('Town')['Sale Amount'].sum().reset_index(name='Total Sale Amount')
    
    merged_data = ct_shp.merge(sales_by_town, how='left', left_on='NAME', right_on='Town')
    
    fig = px.choropleth(merged_data, 
                        geojson=merged_data.geometry, 
                        locations=merged_data.index, 
                        color='Total Sale Amount',
                        hover_name='NAME',
                        title='Total Sale Amount by Town in Connecticut',
                        projection="mercator")
    
    fig.update_geos(fitbounds="locations", visible=False)
    
    # Set the size of the plot
    fig.update_layout(height=600, width=800)  # Adjust the height and width as needed
    
    sales_info = f'Total Sale Amount for {selected_year}: ${total_amount:,.2f}'
    
    return fig, sales_info


In [24]:
@app.callback(
    Output('property-types-distribution', 'figure'),
    [Input('property-types-distribution', 'hoverData')]
)
def display_residential_types_distribution(hoverData):
    if hoverData is None:
        property_types_distribution = df['Property Type'].value_counts().reset_index()
        property_types_distribution.columns = ['Property Type', 'Count']
        fig = px.pie(property_types_distribution, values='Count', names='Property Type', title='Property Types Distribution')
    else:
        hovered_property_type = hoverData['points'][0]['label']
        if hovered_property_type == 'Residential':
            residential_types_distribution = df['Residential Type'].value_counts().reset_index()
            residential_types_distribution.columns = ['Residential Type', 'Count']
            fig = px.pie(residential_types_distribution, values='Count', names='Residential Type', title='Residential Types Distribution')
        else:
            property_types_distribution = df['Property Type'].value_counts().reset_index()
            property_types_distribution.columns = ['Property Type', 'Count']
            fig = px.pie(property_types_distribution, values='Count', names='Property Type', title='Property Types Distribution')
    return fig


In [25]:
# pip install opencage


In [26]:
# Exclude 'Non Use Code' column and 'Residential Type' from dropdown_columns


In [27]:
@app.callback(
    Output('output-graph', 'children'),
    [Input('town-dropdown-last', 'value'),
     Input('property-type-dropdown', 'value')]
)
def update_graph(selected_town, selected_property_types):
    filtered_df = df[
        (df['Town'] == selected_town) &
        (df['Property Type'].isin(selected_property_types))
    ]
    sales_by_year = filtered_df.groupby('List Year')['Sale Amount'].sum().reset_index()
    fig = px.line(sales_by_year, x='List Year', y='Sale Amount', title='Total Sales Over Years')
    return dcc.Graph(figure=fig)

In [28]:
# Run the app
if __name__ == '__main__':
    app.run_server(port=8069,jupyter_mode="external")

Dash app running on http://127.0.0.1:8069/


In [29]:
# df['Property Type'].unique()